# **Tutorial 1: Modelling Heat Transfer in Buildings**

## Overview

In this tutorial, we estimate data-driven heat transfer models for a building envelope. We use real-world measurement data from an occupied, semi-detached house in the UK [1], which was retrofit in Spring/Summer 2016 and occupied in September 2016. The data set is publicly available [here](https://discovery.ucl.ac.uk/id/eprint/10087216/) and comprises two 5-day periods of measurements at a 10-minute resolution.

By the end of this tutorial, you should be able to:
- Process and interpret building measurement data.
- Formulate data-driven (grey- and black-box) models for building heat transfer.
- Compare and validate different modelling approaches using good-of-fit and statistical metrics.

[1] Hollick, F; Wingfield, J; (2018) Two periods of in-situ measurements from an occupied, semi-detached house in the UK [Dataset]. 10.14324/000.ds.10087216.

## Set up

Load the necessary Python libraries.

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['figure.constrained_layout.use'] = True
plt.rcParams['figure.dpi'] = 600
plt.rcParams['figure.figsize'] = (10,4)

## Data processing

We first load and visualize the data. First, set up the ```data_path``` variable to the correct file location and load the data set. The data set spans two 5-day periods. The data from the first period (November 2016) is considered first.

In [4]:
#%% Load measurement data

data_path = 'C:\\Users\\ucbva19\\Git projects\\BENV0092\\data'
data = pd.read_csv(f'{data_path}\\10mins_solpap_2016.csv', index_col = 0)
data.index = pd.to_datetime(data.index, format="%d/%m/%Y %H:%M")

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\ucbva19\\Git projects\\BENV0092\\data\\10mins_solpap_2016.csv'

### Preliminary analysis

The first task is to familiarize ourselves with the data and assess its quality. To familiarize with the data, we can generate plots, provide summary statistics, and estimate correlations/ autocorrelations.

**Discussion**:
- Interpret parameters and estimate some summary statistics.
- Describe some salient characteristics of the ```P_tot (W)``` variable.

In [ ]:
print(data.head())
print(f'Percentage of missing observations: {100*data.isna().sum()/len(data)}')

# Visualization 
fig, ax = plt.subplots(figsize=(10,8), nrows = 3, sharex = True)

data['P_tot (W)'].plot(ax=ax[0])
data[['T_External (degC)', 'T_Average (degC)', 'T_Thermostat (degC)', 'T_Bed1 (degC)', 'T_Living (degC)']].plot(ax=ax[1])
data['Solar (W/m2)'].plot(ax = ax[2])
plt.show()

### Dealing with missing data

Some variables contain missing data. A good practice is to visually examine the missing data and then pick an appropriate imputation method. Considering that we are dealing with 10-minute time series data, a simple linear interpolation seems sensible. 

**Discussion**:
- Consider the most common imputation method, i.e., replacing with the mean value. Is this a good approach here?

In [ ]:
# Fill missing data with linear interpolation
data = data.interpolate('linear')
assert(data.isna().all().sum() == 0)


## Modelling heat transfer

We will formulate different data-driven models for the envelope heat transfer model. The following assumptions are in place:
- We consider a single-zone model and use the average room temperature, ```T_Average (degC)```, as a proxy.
- We consider a 10-minute discretization step, i.e., $dt=1/6$, which is the same as the data granularity.
- We use the total power, ```P_tot (W)```, as a proxy for the net heat injection, which is a simplification. Ideally, we would like to include potential losses. This means that we need to be careful with the physical interpretation of the estimated parameters.

To streamline the code, we introduce auxiliary variables and set up parameters.

In [ ]:
# Lead T_in values (target variable)
data['T_Average_lead_1'] = data['T_Average (degC)'].shift(-1)
data['T_diff'] = data['T_External (degC)'] - data['T_Average (degC)']

# Drop NaNs created from the shift operator
data = data.dropna()

# discretization step at 10 minutes
dt_h = 1/6

### Deriving the $RC$ model

We consider a simple $1R-1C$ model to approximate the heat transfer dynamical model of a single-zone dwelling.:

- $R_{\text{th}}$ = thermal resistance (°C/kW)  
- $C_{\text{th}}$ = thermal capacitance (kWh/°C)
- $\tau = R_{\text{th}} C_{\text{th}}$ = time constant

The continuous-time energy balance is given by

<center>$ C_{\text{th}} \frac{dT}{dt} = \frac{T_\text{ext} - T}{R_{\text{th}}} + H_\text{heat} + g H_\text{solar}$,</center>


where $T$ is the average indoor temperature (°C), $T_\text{ext}$ is the external temperature (°C), and $H_\text{heat}$ is the sensible heat injection in the node (kW), $H_\text{solar}$ is the solar radiation, and $g$ is the (unknown) solar gain factor.

Equivalently, we can write 
<center>$ \frac{dT}{dt} =
\frac{1}{\tau}(T_\text{ext} - T)
+ \frac{1}{C_{\text{th}}}H_{\text{heat}} + \frac{g}{C_{\text{th}}}H_{\text{solar}}$.</center>

Using forward Euler discretization, we get
<center>
$
T_{k+1}
=
T_k
+ \Delta t \frac{1}{\tau}(T_{\text{ext},k} - T_k)
+ \Delta t \frac{1}{C_{\text{th}}} H_{\text{heat},k} + \Delta t \frac{g}{C_{\text{th}}} H_{\text{solar},k},
$    
</center>


where $k$ is the time step index. For convenience, we define the **discrete coefficients**:

<center>
$
\alpha_1 = \frac{\Delta t}{\tau}, \qquad
\alpha_2 = \frac{\Delta t}{C_{\text{th}}}, \qquad,
\alpha_3 = \frac{\Delta t g}{C_{\text{th}}}.
$    
</center>

The final discrete $RC$ model is given by

<center>
$
T_{k+1} = T_k + \alpha_1 (T_{\text{out},k} - T_k)
+ \alpha_2 H_{\text{heat},k} + \alpha_3 H_{\text{solar},k}.$
</center>


### $RC$ parameter estimation

Next, we estimate the parameters of the $RC$ model, or, equivalently, coefficients $\alpha_{1,2,3}$. Throughout, we use $\hat{\cdot}$ to denote parameters estimated (i.e., learned) from data. First, for the $k$th observation, we define the loss function

<center>
$\ell(\hat{T}_k, T_{k})=(\hat{T}_k-T_{k})^2 = \bigl(T_k + \alpha_1(T_{\text{out},k} - T_k)
+ \alpha_2 H_{\text{heat},k} + \alpha_3 H_{\text{solar},k} - T_{k+1}\bigr)^2$,
</center> 

which represents the mean squared error (MSE) between predicted temperature $\hat{T}$ and actual temperature $T$.
To estimate the parameters, we minimize the empirical mean squared error over the training data set which contains $N$ observations, 
given by

<center>
$\min_{\alpha_{1,2,3}} \sum_{k=1}^{N-1} \ell(\hat{T}_k, T_{k})$,    
</center>
 
which defines a constrained regression problem.

**Excercise**: Your task is to solve the constrained optimization problem described above and return the $RC$ parameters. 

*Hint*: You can use the ```cvxpy``` library to formulate and solve a constrained optimization problem. The ```lsq_linear``` function can also be used to solve a constrained linear least squares problem.


In [ ]:
#%% Find parameters for lumped RC model

Y = data['T_Average_lead_1']
X = data[['T_diff', 'P_tot (W)', 'Solar (W/m2)', 'T_Average (degC)']]

# Solve a constrained least-squares optimization problem to estimate RC parameters

# Estimate in-sample Root Mean Squared Error
print('In-sample RMSE')

# Return parameters tau, C, R, g
print('RC parameters')

# Add a visualization


### Linear regression

We now model the heat transfer as an (unconstrained) linear regression given by $f(x;\theta) = \theta^\top x$, where $x$ concats all the features used (```['T_diff', 'P_tot (W)', 'Solar (W/m2)', 'T_Average (degC)']```) and $\theta$ are the coefficients. To train the linear regression (i.e., to learn parameters $\theta$) we need to solve

<center>
$\min_{\theta} \sum_{k=1}^{N-1} \ell(f(x;\theta), T_{k})$,    
</center>

which is a standard least squares regression.  

In [ ]:
# Redefine feature vector
Y = data['T_Average_lead_1']
X = data[['T_diff', 'P_tot (W)', 'Solar (W/m2)', 'T_Average (degC)']]

# Fit a linear regression model

# Solve a constrained least-squares optimization problem to estimate RC parameters

# Estimate in-sample Root Mean Squared Error
print('In-sample RMSE')

# Return parameters tau, C, R, g
print('RC parameters')

# Add a visualization

**Discussion**:
- Which model has the lowest RMSE? Could you have guessed it beforehand?
- Try to interpret RC parameters from unconstrained linear regression model. What happens? Do the values make sense?

**Additional exercises**:
- Estimate $R^2$ and other goodness-of-fit measures.
- Fit a nonlinear machine learning model, e.g., random forecasts, gradient boosting.
- Add more features.

## Repeating the experiment using June 2017 data

Previous experiment used data from November 2016. We repeat the experiment using measurement data from June 2017. 

**Discussion:**
- What do you observe? How are the estimated parameters affected? Is the fit better or worse?

In [ ]:
#%% Load measurement data from 2017
data_17 = pd.read_csv(f'{data_path}\\10mins_solpap_2017.csv', index_col = 0)
data_17.index = pd.to_datetime(data_17.index, format="%d/%m/%Y %H:%M")

# Fill missing data with linear interpolation
data_17 = data_17.interpolate('linear')
assert(data_17.isna().all().sum() == 0)

# Lead T_in values (target variable)
data_17['T_Average_lead_1'] = data_17['T_Average (degC)'].shift(-1)
data_17['T_diff'] = data_17['T_External (degC)'] - data_17['T_Average (degC)']

# Drop NaNs created from the shift operator
data_17 = data_17.dropna()

# Set discretization step to 10 minutes
dt_h = 1/6

# Fill missing data with linear interpolation
data_17 = data_17.interpolate('linear')
assert(data_17.isna().all().sum() == 0)

# Visualization 
fig, ax = plt.subplots(figsize=(10,8), nrows = 3, sharex = True)

data_17['P_tot (W)'].plot(ax=ax[0])
data_17[['T_External (degC)', 'T_Average (degC)', 'T_Thermostat (degC)', 'T_Bed1 (degC)', 'T_Living (degC)']].plot(ax=ax[1])
data_17['Solar (W/m2)'].plot(ax = ax[2])
plt.show()

###########%% Find parameters for lumped RC model

Y = data_17['T_Average_lead_1']
X = data_17[['T_diff', 'P_tot (W)', 'Solar (W/m2)', 'T_Average (degC)']]

# Estimate in-sample Root Mean Squared Error
print('In-sample RMSE')

# Return parameters tau, C, R, g
print('RC parameters')

# Add a visualization
